# <img src="https://img.icons8.com/bubbles/100/000000/3d-glasses.png" style="height:50px;display:inline"> EE 046746 - Technion - Computer Vision


# Tutorial 01 - Feature Detection
---
<img src="./assets/LineDetection.png" width="800">

* <a href="https://towardsdatascience.com/tutorial-build-a-lane-detector-679fd8953132">Image source</a>

## <img src="https://img.icons8.com/bubbles/50/000000/checklist.png" style="height:50px;display:inline"> Agenda

* Reminder - Derivatives and Edges
    * Image Gradient
* Corner Detection
    * Why Corners?
    * Harris Corner Detector
    * Harris Conclusion With Vectorized Operations
    * Invariance to Image Deformation
    * Laplacian Feature Detector
* Multi-scale Detection
    * Multi-scale Laplacian
    * Multi-scale With Harris
* Recommended Videos
* Credits


## <img src="https://img.icons8.com/nolan/64/edge-level.png" style="height:50px;display:inline"> Reminder - Derivatives and Edges

---
* An edge is a place of rapid change in the image intensity function.

<img src="./assets/tutorial_1/derive1.JPG" width="600">


#### <img src="https://img.icons8.com/nolan/64/edge-level.png" style="height:50px;display:inline"> Image Gradient
---
* The gradient of an image: $\nabla f= [\frac{\partial f}{\partial x},\frac{\partial f}{\partial y}]$
* The gradient points in the direction of most rapid change in intensity 

<img src="./assets/derive2.JPG" width="600">

* The gradient direction (orientation of edge normal) is given by: $\theta = tan^{-1} (\frac{\partial f}{\partial y}/\frac{\partial f}{\partial x})$

* The edge strength is given by the gradient magnitude
$||\nabla f|| = \sqrt {(\frac{\partial f}{\partial y})^2+(\frac{\partial f}{\partial x})^2}$

Image derivatives can be implemented as a convolution:

<img src="./assets/tutorial_1/image_derivatives.png" style="height:600px">

## <img src="https://img.icons8.com/nolan/64/edge-level.png" style="height:50px;display:inline"> Corner Detection

Although edges are good for human perception, and contain most of the information, they are not allways good enough. Another, more useful building block for many computer vision applications is a corner detector. Corners are better features for tasks such as:
* Image alignment (homography, fundamental matrix)
* 3D reconstruction
* Motion tracking
* Object recognition
* Indexing and database retrieval
* Robot navigation

<img src="./assets/tutorial_1/planar_instance_recognition.png" style="height:300px"> <img src="./assets/tutorial_1/image_matching.png" style="height:300px">

<img src="./assets/tutorial_1/3d_obj_recognition.png" style="height:300px"> <img src="./assets/tutorial_1/robot_localization.png" style="height:300px">

### <img src="https://img.icons8.com/nolan/64/edge-level.png" style="height:50px;display:inline"> Why Corners?

suppose we select the yellow window, which contains an edge, as a feature. we want to find it in an image from a different angle:

<img src="./assets/tutorial_1/grafity_bad_feature.png" style="height:300px">

Where is the match?

<img src="./assets/tutorial_1/grafity_bad_feature2.png" style="height:300px">

There are multiple windows with similar content, just by looking at the local content we won’t be able to tell which one matches.


Let's see another example. Pick a point for a feature in the following image:

<img src="./assets/tutorial_1/Trapez.png" style="height:200px">

Now find it again in the next image:

<img src="./assets/tutorial_1/Trapez_rotated.png" style="height:200px">

What type of feature would be a good selection? Corners of course!
Corners are easily recognised by looking at a small window. Shifting the window in any direction should give large change in intensity.

<img src="./assets/tutorial_1/flat_edge_corner.png" style="height:400px">

### <img src="https://img.icons8.com/nolan/64/edge-level.png" style="height:50px;display:inline"> Harris Corner Detector

Given a window $W$ of $nxn$ pixels, we perform the following steps to determine whether it’s a corner:
1.	Compute image gradients over small region
2.	Subtract the mean from each image gradient
3.	Compute the covariance matrix
4.	Compute eigenvectors and eigenvalues
5.	Use threshold on eigenvalues to detect corners

We slide a window through the image and perform all of these steps for each window.
We will discuss each step separately.

### 1. Compute image gradients over a small region

For our window $W$, we compute the derivative in each direction (x and y), which generates two $nxn$ arrays $W_x, W_y$

<img src="./assets/tutorial_1/window_derivatives.png" style="height:400px">

Recall that we get high response in $W_x, W_y$ if there is an edge on the corresponding direction. 

<img src="./assets/tutorial_1/derivatives_example.png" style="height:300px">

*Note: Since we are going to perform this process on a sliding window, we are going to need to compute the gradient at all points any way.In practice it will be more efficient to compute the gradients for the entire image and extract the current window from the derivatives image.* 


For each pixel $i,j$ in the window, we have a gradient vector $(W_x^{i,j}, W_y^{i,j})$. We would like to look at the distribution of these gradients for the window. We can plot the $n^2$ gradients on a 2d plane, let's see some examples:

<img src="./assets/tutorial_1/grad_distribution.png" style="height:500px">

on the left pach, we have a flat region, therefore the derivative in both directions are small, and most of the points will be around the origin (assume there is some noise, that's why gradients are not exactly zero)

on the central patch, we have a vertical edge, therefore the $x$ derivative is high on some points, we get a distribution around the x axis.

on the right patch, there are some pixels with vertical only gradient and some pixels with a diagonal gradient, so we get points spread along two lines.

2D plot are nice, but we want a way to quantify the results and determine if there's a corner in the patch. We are going to do that by looking at the covariance matrix of the gradients.

### 2. Subtract the mean from each image gradient
To define the covariance we first compute the mean of the data and subtract it, so that the data is centered around 0. 
<img src="./assets/tutorial_1/subtract_mean.png">

### 3. Compute the covariance matrix
For a general distribution, given a set of $N$ 2D vectors   $v^1,v^2,…$ (after mean subtraction) the covariance is defined as

$$\Sigma = \frac{1}{N} \sum_{j=1}^{N} v^j {v^j}^T 
 = \frac{1}{N} \begin{bmatrix}
 \sum_{j=1}^{N} v^j_x v^j_x  & \sum_{j=1}^{N} v^j_x v^j_y \\
\sum_{j=1}^{N} v^j_x v^j_y & \sum_{j=1}^{N} v^j_y v^j_y
\end{bmatrix}$$

Note that there are 3 sums in the covariance matrix. In our case this covariance matrix is:

$$\Sigma = \frac{1}{N} \sum_{i,j \in 1...n} g^j {g^j}^T
 = \frac{1}{n^2} \begin{bmatrix}
 \sum_{i,j \in 1...n} {W_x^{i,j}}^2 & \sum_{i,j \in 1...n} W_x^{i,j}W_y^{i,j}\\
\sum_{i,j \in 1...n} W_x^{i,j}W_y^{i,j} & \sum_{i,j \in 1...n} {W_y^{i,j}}^2
\end{bmatrix}$$

The sums can be computed the following way:

<img src="./assets/tutorial_1/cov_matrix_sums.png" style="width:600px">

where .* means matrix elementwise multipication.

### 4. Compute Eigenvalues 

#### Reminder
Since the covariance matrix is symetric (and PSD = Positive Semi Defined), it can be decomposed:
$$\Sigma = R^T \Lambda R = 
\begin{bmatrix}
-\bar{r_1}^T- \\
-\bar{r_2}^T-
\end{bmatrix}
\begin{bmatrix}
\lambda_1  & 0\\
0 & \lambda_2
\end{bmatrix}
\begin{bmatrix}
\bar{r_1} & \bar{r_2} \\ 
| & |
\end{bmatrix}
$$

For a symetric PSD matrix this is called SVD - Singular Value Decomposition. 
Where $\lambda_1$ and $\lambda_2$ are the eigenvalues, and R is an orthogonal matrix (can say it's a rotation matrix), built from orthogonal eigenvectors. 

Note that in this decomposition $R^{-1} = R^T$

In a 2D covariance matrix, the eigenvectors point to the directions with the highest and lowest variance. The eigenvalues are the corresponding variances.
<img src="./assets/tutorial_1/covariance_eigens.png">

#### Back to corners
In the decomposition $\Sigma = R^T \Lambda R $ the eigenvectors (columns of R) are the directions where there is maximal and minimal variance in the gradients of the window, and the corresponding eigenvalues are the variance. That means that if we rotate the patch, only R will change, but the eigenvalues will remain the same.

We want to use the eigenvalues for corner detection, this will also promise us that our detector will be invariant to rotations. 

<img src="./assets/tutorial_1/corner_rotation.png">



In a corner, both eigenvalues should be high

<img src="./assets/tutorial_1/corner_edge_variance.png">

### 5. Use threshold on eigenvalues to detect corners

We have the eigenvalues, and we want to decide whether our patch is a corner. Where do we want $(\lambda_1, \lambda_2)$ to be?

<img src="./assets/tutorial_1/lambdas_plane.png" style="height:300px">

* In a flat window, we have low variance in any direction, which means both lambda's will be close to zero.
* In an edge, we have high variance on one direction but low variance on the orthogonal direction, resulting one lambda high and one low.
* In a corner, we have high variance in two directions (our ellipse will become a large circle), both lambda will be high. 

<img src="./assets/tutorial_1/lambdas_plane_with_patches.png" style="height:300px">

**Clarification: we can get $\lambda_1<<\lambda_2$ or $\lambda_2<<\lambda_1$ in a non vertical or horizontal edge as well. If we have a diagonal edge, one eigenvalue will represent the variance on it's direction and one on the orthogonal direction. This is due to how the SVD works.**

Can you think of a good score function $R(\lambda_1, \lambda_2)$ for a window?

<img src="./assets/tutorial_1/lambdas_strong_corner.png" style="height:300px">

One such a function we can use is the min eigenvalue. If it is high then we have a corner. 
$$ R = min(\lambda_1, \lambda_2)$$

<img src="./assets/tutorial_1/lambdas_min.png" style="height:300px">

*Note: In some formulations the SVD decompoziation is defined such that the eigenvalues are in descending order, this way we can have threshold only on $\lambda_2$*

The problem with the minimum function, is that we need to know the eigenvalues. performing the computation is quite expensive. Recall that we have to do it many times on a sliding window. We can use some tricks to find a cheaper score function.

Recall that the sum of eignevalues of matrix equals to it's trace, and their multipications to it's determinant. for 2x2 matrix:
$$\Sigma = \begin{bmatrix} \sigma_1 &\sigma_2 \\ \sigma_3 & \sigma_4 \end{bmatrix}$$

$$ det(\Sigma) = \lambda_1 \lambda_2 = \sigma_1 \sigma_4 - \sigma_2 \sigma_3 \\
trace(\Sigma) = \lambda_1 + \lambda_2 = \sigma_1 + \sigma_4 $$

We don't have the eigenvalues, but we have their multipication and sum. we can define another, less exprensive to compute, score function:
$$R = \lambda_1 \lambda_2 - \kappa (\lambda_1 + \lambda_2)^2 \\ \kappa < 0.25$$

Let's look at the function, intuitively
* If $\lambda_1=\lambda_2=0$ we get R = 0
* If $\lambda_1>0, \lambda_2=0$, then $\lambda_1 \lambda_2=0, (\lambda_1+\lambda_2)^2>0 $ we get R<0
* If $\lambda_1=\lambda_2>0$ we get $R = \lambda_1^2 (1-4\kappa) > 0 $

So this score is high when both eigenvalues are high

Here is an example with $\kappa = 0.2$ and different thresholds for the score:

<img src="./assets/tutorial_1/desmos_scores.png" style="height:500px">

https://www.desmos.com/calculator/oi9q42yvt4

### <img src="https://img.icons8.com/nolan/64/edge-level.png" style="height:50px;display:inline"> Harris Conclusion With Vectorized Operations
When working with images we want as match as possible to define our operations with matrices. This is a more efficient way to implement with high level programming languages, and this is the only way to make use of computation accelerators.

Let's conclude the Harris Detector, with more efficient vector operations. Given an image $I$:

#### 1
Compute x and y derivatives $$I_x = G^x * I$$ $$I_y = G^y * I$$ $G^x$ and $G^y$ are gaussian derivatives filters, which perform the blur and derivation simultaneously.

#### 2

compute the product of the derivatives for every pixel $$I_{x^2} = I_x \cdot I_x $$ $$I_{y^2} = I_y \cdot I_y $$  $$I_{xy} = I_x \cdot I_y $$
(we use elementwise product here)

#### 3
compute the sum of the products for a window around each pixel. That can also be done with convolution with a gaussian blur filter to remove noise.
    $$ S_{x^2} = G_\sigma * I_{x^2}$$
    $$ S_{y^2} = G_\sigma * I_{y^2}$$
    $$ S_{xy} = G_\sigma * I_{xy}$$
#### 4
Compute the score for each window. The score is defined by the eigenvalues of the covariance matrix:
    $$\Sigma(x,y) = \begin{bmatrix} S_{x^2}(x,y) & S_{xy}(x,y) \\ S_{xy}(x,y) & S_{y^2}(x,y) \end{bmatrix}$$
    but we don't need to compute it for the score, just the determinant and trace for each x,y. We can do it with one vectorized operation for all  pixels. $ \overline{\overline{R}} $ will be a 2d image with the score for each pixel (and the window around it). 
    $$\overline{\overline{R}} = det(\Sigma) - \kappa(trace(\Sigma))^2 = S_{x^2}S_{y^2} - S_{xy}^2 - \kappa (S_{x^2} + S_{y^2})^2$$ 

#### 5
threshold on value of R to determine where are the corners.

### <img src="https://img.icons8.com/nolan/64/edge-level.png" style="height:50px;display:inline"> Invariance to Image Deformation

Recall that we want to detect the same corner features in different images of the same object. These images may undergo various deformations:
* Change in view point
* Zoom (scale)
* Illumination
* Rotation
* ...

We already know that Harris Detector is invariant to  rotation, since eigenvalues of the covariance matrix dont change under rotation, just the rotation matrix (the eigenvectors).

What about illumination? Illumination usually is descrived as affine intensity change. 

for an image $I(x,y)$ we could define such transformation with parameters $a,b$ as $I'(x,y) = aI(x,y) + b$.

Since Harris uses derivatives of the image, it won't be affected by the constant intensity change b, but it's not invariant to the change in scale caused by a. 

What about scale? Harris is **not** invariant to scale!

<img src="./assets/tutorial_1/scale_variance.png" style="height:200px">

We will soon see what can we do about it.

## <img src="https://img.icons8.com/nolan/64/edge-level.png" style="height:50px;display:inline"> Laplacian Feature Detector
Ther laplacian of an image is a version of a "scalar multidimensional second derivative". Given an image (or a window) $I(x,y)$ the laplacian is defined as:
$$ L(I) = \frac{\partial^2 I}{\partial x^2} + \frac{\partial^2 I}{\partial y^2}$$ 

Laplacian filter is often combined with gaussian filter to blur the image. Applying gaussian blur and then laplacian on an image, is equivalent to applying the laplacian on the gaussian, and then convolve the resulted filter on the image. This way we get the LoG - Laplacian of Gaussian filter.

<img src="./assets/tutorial_1/LoG.png" style="height:200px">

Instead of computing the second derivative of the gaussian, it is simpler and more efficient (we will see why soon) to just subtract two gaussians with different variances to approximate it.

<img src="./assets/tutorial_1/DoG.png" style="height:200px">

This filter have high response for blobby circular image areas. 

<img src="./assets/tutorial_1/sunflowers_laplacian.png" style="height:500px">

The laplacian, like the Harris Detector, is affected by scale. a laplacian filter with different sigmas will have high response for different patches depending on their scale. Note the difference for several variances, on the same image with different scale:

<img src="./assets/tutorial_1/sunflowers_response1.png" style="width:400px"> <img src="./assets/tutorial_1/sunflowers_response2.png" style="width:400px">


<img src="./assets/tutorial_1/sunflowers_response3.png" style="width:400px"> <img src="./assets/tutorial_1/sunflowers_response4.png" style="width:400px">

<img src="./assets/tutorial_1/sunflowers_response5.png" style="width:400px"> <img src="./assets/tutorial_1/sunflowers_response6.png" style="width:400px">

Response is highest when signal has the same characteristic scale as the filter.

<img src="./assets/tutorial_1/laplacian_response.png" style="width:800px">

## <img src="https://img.icons8.com/nolan/64/edge-level.png" style="height:50px;display:inline"> Multi-scale Detection
We are going to see multi-scale detection on Laplacian (DoG) feature detector. Same ideas can be applied on the Harris Detector, but the laplacian is a bit easier to visualize.

### <img src="https://img.icons8.com/nolan/64/edge-level.png" style="height:50px;display:inline"> Multi-scale Laplacian
How can we make a feature detector scale-invariant?
We can look for features in multiple scales, and find local maxima in both position and scale.

To compute multiple laplacians for an image, we first compute gaussians with different scales (variances), and then subtract each gaussian image from it's neighbor. We get multiple laplacians images with different scales. This is called DoG pyramid.

<img src="./assets/tutorial_1/DoG_pyramid.png">


In normal Laplacian Detector, we would have chosen features at points with local maximal response to the filter in space (x and y).

In multi-scale laplacian, we will choose maximal response in space and scale (x,y and scale):

<img src="./assets/tutorial_1/multi_scale_max.png">

<img src="./assets/tutorial_1/sunflower_max_response.png">

#### Filtering Edge Features
Laplacian filter has high response to circles, but it also has quite a high response for straight lines and edges, which we don't want as features, although they are local maximum points.  

<img src="./assets/tutorial_1/butterfly_features.png">

Detection is usually followed by cleaning stage to maintain only informative features. Usually using techniques similar to those we saw in Harris Detector: look at eigenvalues of the covariance matrix of local derivatives.

### <img src="https://img.icons8.com/nolan/64/edge-level.png" style="height:50px;display:inline"> Multi-scale With Harris

We have seen multi-scale detection of Laplacian (difference of Gaussian) features. 
One can also define a Harris corner detector in a multi-scale way: change the scale of the derivative filter and change the size of the window over which we average gradients.  We can then look for local extrema in the scale space.
For various historical reasons, the popular SIFT descriptor you will see in the lecture detects keypoints using difference of Gaussians rather than Harris.
 



### <img src="https://img.icons8.com/bubbles/50/000000/video-playlist.png" style="height:50px;display:inline"> Recommended Videos
---
#### <img src="https://img.icons8.com/cute-clipart/64/000000/warning-shield.png" style="height:30px;display:inline"> Warning!
* These videos do not replace the lectures and tutorials.
* Please use these to get a better understanding of the material, and not as an alternative to the written material.

#### Video By Subject
* Edge Detection (Sobel + Canny) -  <a href="https://www.youtube.com/watch?v=07qwMn2c5iY"> CSCI 512 - Lecture 09-1 Edge Detection </a> | <a href="https://www.youtube.com/watch?v=PUcbhaGhgtE"> CSCI 512 - Lecture 09-2 Edge Detection </a>
    * Edge Detection (Sobel) - <a href="https://www.youtube.com/watch?v=uihBwtPIBxM"> Finding the Edges (Sobel Operator) - Computerphile </a>
    * Edge Detection (Canny) - <a href="https://www.youtube.com/watch?v=sRFM5IEqR2w"> Canny Edge Detector - Computerphile </a>- 
    

## <img src="https://img.icons8.com/dusk/64/000000/prize.png" style="height:50px;display:inline"> Credits
----
* Slides - Anat Levin
* EE 046746 Spring 2020 - <a href="https://il.linkedin.com/in/dahlia-urbach-97a816123">Dahlia Urbach</a>
* Tutorial: Build a lane detector - <a href="https://towardsdatascience.com/tutorial-build-a-lane-detector-679fd8953132">Chuan-en Lin</a>
* Icons from <a href="https://icons8.com/">Icon8.com</a> - https://icons8.com